In [1]:
def add_to_class(Class):
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
    return wrapper

In [1]:
import tensorflow as tf
print('Tensorflow version:', tf.__version__)

from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

Tensorflow version: 2.15.0


Load and prepare the MNIST dataset

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255., x_test/255.

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype('float32')
x_test = x_test[..., tf.newaxis].astype('float32')

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
x_train.shape

(60000, 28, 28, 1)

Using `tf.data` to batch and shuffle the dataset

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)
).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [5]:
x, y = next(iter(train_ds))
print(x.shape, y)

(32, 28, 28, 1) tf.Tensor([8 3 7 9 2 2 7 7 7 2 1 7 8 8 8 2 8 5 2 3 6 7 5 6 7 0 2 4 0 2 5 7], shape=(32,), dtype=uint8)


Build the `tf.keras` model using the Keras model subclassing API

In [6]:
class MyModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation = 'relu')
        self.flatten = keras.layers.Flatten()
        self.d1 = keras.layers.Dense(128, activation = 'relu')
        self.d2 = keras.layers.Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

Choose an optimizer and loss function for training

In [7]:
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits = True)
optimizer = keras.optimizers.Adam()

Select metrics to measure the loss and accuracy of the model. These metrics accumulate the values over epochs and then print the overal result

In [8]:
train_loss = keras.metrics.Mean(name = 'train_loss')
train_acc = keras.metrics.SparseCategoricalAccuracy(name = 'train_acc')

test_loss = keras.metrics.Mean(name = 'test_loss')
test_acc = keras.metrics.SparseCategoricalAccuracy(name = 'test_acc')

Use `tf.GradientTape` to train the model

In [9]:
model = MyModel()

In [10]:
model.build(input_shape = (None, 28, 28, 1))
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  320       
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  2769024   
                                                                 
 dense_1 (Dense)             multiple                  1290      
                                                                 
Total params: 2770634 (10.57 MB)
Trainable params: 2770634 (10.57 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    #loss_ = keras.losses.categorical_crossentropy(labels, predictions, from_logits = True)
    loss_ = loss_object(labels, predictions)
  gradients = tape.gradient(loss_, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss_)
  train_acc(labels, predictions)

@tf.function
def test_step(images, labels):
    # training = False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout)
    predictions = model(images, training = False)
    #loss_ = keras.losses.categorical_crossentropy(labels, predictions, from_logits = True)
    loss_ = loss_object(labels, predictions)

    test_loss(loss_)
    test_acc(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_state()
    train_acc.reset_state()
    test_loss.reset_state()
    test_acc.reset_state()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result():0.2f}, '
        f'Accuracy: {train_acc.result() * 100:0.2f}, '
        f'Test Loss: {test_loss.result():0.2f}, '
        f'Test Accuracy: {test_acc.result() * 100:0.2f}'
    )

Epoch 1, Loss: 0.13, Accuracy: 95.98, Test Loss: 0.06, Test Accuracy: 98.12
Epoch 2, Loss: 0.04, Accuracy: 98.72, Test Loss: 0.05, Test Accuracy: 98.13
Epoch 3, Loss: 0.02, Accuracy: 99.25, Test Loss: 0.06, Test Accuracy: 98.12
Epoch 4, Loss: 0.01, Accuracy: 99.63, Test Loss: 0.06, Test Accuracy: 98.22
